## Tutorial showing how to create Parcels in Agulhas animated gif

This brief tutorial shows how to recreate the [animated gif](http://oceanparcels.org/animated-gifs/globcurrent_fullyseeded.gif) showing particles in the Agulhas region south of Africa.

We start with importing the relevant modules

In [1]:
from parcels import FieldSet, ParticleSet, JITParticle, AdvectionRK4, ErrorCode
from datetime import timedelta
import numpy as np

INFO: Compiled ParcelsRandom ==> /tmp/parcels-40642/libparcels_random_bc2987a5-ca79-4f71-8595-ecdef368b180.so


Now load the Globcurrent fields from the `GlobCurrent_example_data` directory (note that unlike in the main Parcels tutorial we don't use a dictionary for the filenames here; as they are the same for all variables, we don't need to)

In [2]:
filenames = "/contrib/Stefan.Gary/jupyterlab/parcels_examples/GlobCurrent_example_data/20*.nc"
variables = {'U': 'eastward_eulerian_current_velocity',
             'V': 'northward_eulerian_current_velocity'}
dimensions = {'lat': 'lat',
              'lon': 'lon',
              'time': 'time'}
fieldset = FieldSet.from_netcdf(filenames, variables, dimensions)

Now create vectors of Longitude and Latitude starting locations on a regular mesh, and use these to initialise a `ParticleSet` object.

In [3]:
lons, lats = np.meshgrid(range(15, 35), range(-40, -30))
pset = ParticleSet(fieldset=fieldset, pclass=JITParticle, lon=lons, lat=lats)

Now we want to advect the particles. However, the Globcurrent data that we loaded in is only for a limited, regional domain and particles might be able to leave this domain. We therefore need to tell Parcels that particles that leave the domain need to be deleted. We do that using a `Recovery Kernel`, which will be invoked when a particle encounters an `ErrorOutOfBounds` error:

In [4]:
def DeleteParticle(particle, fieldset, time):
    particle.delete()

Now we can advect the particles. Note that we do this inside a `for`-loop, so we can save a plot every six hours (which is the value of `runtime`). See the [plotting tutorial](http://nbviewer.jupyter.org/github/OceanParcels/parcels/blob/master/examples/tutorial_plotting.ipynb) for more information on the `pset.show()` method.

In [5]:
# Inputs: pset, DeleteParticle
# Outputs: list of files (each file name is in savefile)
for cnt in range(3):
    # Set filename for output plot
    output_image = 'particles'+str(cnt).zfill(2)
    print(output_image)
    
    # First plot the particles
    pset.show(savefile=output_image, field='vector', land=True, vmax=2.0)

    # Then advect the particles for 6 hours
    pset.execute(AdvectionRK4,
                 runtime=timedelta(hours=6),  # runtime controls the interval of the plots
                 dt=timedelta(minutes=5),
                 recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle})  # the recovery kernel

particles00


/contrib/Stefan.Gary/miniconda3/envs/oceanparcels/lib/python3.8/site-packages/parcels/plotting.py:191: RuntimeWarning: invalid value encountered in true_divide
  u = np.where(speed > 0., data[0]/speed, 0)
/contrib/Stefan.Gary/miniconda3/envs/oceanparcels/lib/python3.8/site-packages/parcels/plotting.py:192: RuntimeWarning: invalid value encountered in true_divide
  v = np.where(speed > 0., data[1]/speed, 0)
INFO: Plot saved to particles00.png
INFO: Compiled JITParticleAdvectionRK4 ==> /tmp/parcels-40642/11dcc244d2eeaaa77d5025d504e0b49f_0.so


particles01


INFO: Plot saved to particles01.png


particles02


INFO: Plot saved to particles02.png


This now has created 3 plots. Note that the original animated gif contained 20 plots, but to keep running of this notebook fast we have reduced the number here. Of course, it is trivial to increase the number of plots by changing the value in the `range()` in the cell above.

As a final step, you can use [ImageMagick](http://www.imagemagick.org/script/index.php) or an online tool to stitch these individual plots together in an animated gif.

# Execute the above in parallel on remote, custom workers

+ Run OceanParcels in 4 separate experiments at the same time.
+ Each experiment is defined by a Python/OceanParsels script.
+ One experiment per (customizable) worker node.
+ Each instance of OceanParcels is run via a Docker container.

In [6]:
import parsl
from parsl.app.app import python_app, bash_app
from parsl.data_provider.files import File
from path import Path
import os
#from parsl.configs.local_threads import config
#from parslpw import pwconfig,pwargs

prefix = ''
from parsl.config import Config
from parsl.executors.threads import ThreadPoolExecutor
from parsl.executors import HighThroughputExecutor
from parsl.providers import SlurmProvider
config = Config(
    executors=[
        HighThroughputExecutor(
            label='slurm',
            worker_debug=False,
            cores_per_worker=int(1),
            working_dir = '/tmp/pworks/',
            worker_logdir_root = os. getcwd() + '/parsllogs',
            provider = SlurmProvider(
                partition = 'compute',
                nodes_per_block = 1,
                min_blocks = int(0),
                max_blocks = int(10),
                parallelism = 1 # Was 0.80 ### # FIXME?,
            )
        )
    ]
)

parsl.load(config)

print("pwconfig loaded")

pwconfig loaded


In [7]:
@bash_app
def run_ocean_parcels(stdout='ocean_parcels.stdout', 
                      stderr='ocean_parcels.stderr', inputs=[], outputs=[]):
    
    import os
    run_script = os.path.basename(inputs[0])
    container_run = os.path.basename(inputs[1])
    out_file = os.path.splitext(os.path.basename(inputs[0]))[0]+'.gif'
    out_dir = os.path.basename(outputs[0])
    
    run_command = "/bin/bash " + container_run + " " + run_script
    
    # The text here is interpreted by Python (hence the %s string substitution
    # using strings in the tuple at the end of the long text string) and then
    # run as a bash app.

    return '''
        outdir=%s
        outfile=%s
        mkdir -p $outdir
        cd $outdir
        cp ../%s ./
        cp -sv ../%s ./
        %s
        mv movie.gif $outdir/$outfile
    ''' % (out_dir,out_file,run_script,container_run,run_command)

In [ ]:
LOCAL_TESTING = True
if LOCAL_TESTING:
    import argparse
    pwargs = argparse.Namespace()
    pwargs.out_dir = '/contrib/Stefan.Gary/jupyterlab/ocean-parcels/test-outputs'
    pwargs.run_files = '/contrib/Stefan.Gary/jupyterlab/ocean-parcels/test_ocean_parcels_1.py---/contrib/Stefan.Gary/jupyterlab/ocean-parcels/test_ocean_parcels_2.py---/contrib/Stefan.Gary/jupyterlab/ocean-parcels/test_ocean_parcels_3.py---/contrib/Stefan.Gary/jupyterlab/ocean-parcels/test_ocean_parcels_4.py'

wrapper = Path("/contrib/Stefan.Gary/jupyterlab/ocean-parcels/wrap_singularity_oceanparcels_slurm.sh")
run_files = pwargs.run_files.split('---')

runs=[]
for run_file_name in run_files:
    
    # An Ocean Parcels script
    run_file = Path(run_file_name)
    
    # Output directory based on script name
    out_dir = Path(os.path.splitext(os.path.basename(run_file_name))[0])
    
    # Run the containers in parallel in separate directories
    r = run_ocean_parcels(inputs=[run_file,wrapper], 
                          outputs=[out_dir])
    
    runs.append(r)

print("Running",len(runs),"OceanParcels executions...")
[r.result() for r in runs]

Running 4 OceanParcels executions...
